# Data Preparation for loading into bd

### Назначение: Подготовка данных, скрапинг, парсинг и создание задания в виде excel файла для загрузки на сайт.

[API Редактор інструкцій - ссылка на сайт](http://185.174.220.122)

-----------

### Импорт и проверка констант:

In [1]:
import os
import sys
from pprint import pprint
from icecream import ic

import pandas as pd
# подгрузка root
sys.path.append('..')
import settings

from core.utilities import read_excel_package, is_open_file, DFS_TYPE, dfs_to_excel, read_excel

from apteka911.service_utilities.one_drive_txt_file_to_df import transform_to_df
from apteka911.service_utilities.path_finder import search_replace
from apteka911.service_utilities.symptoms_creator import create_symptoms_dfs, SYMPTOMS_PATH

from apteka911.page_scraping import scrape_pages, DOMAIN_URL, SOURCE_HTML_DIR
from apteka911.page_parsing import parse_pages, PARSED_DATA_PATH
from apteka911.task_creator import create_task_dfs, TASK_DFS_PATH

%load_ext dotenv
%dotenv

### Проверить значения IP из .env:

In [ ]:
API_URL = os.getenv("API_URL")
print(f"{API_URL=}")

#### Проверить путь к папке с заданиями в txt файлах:

In [ ]:
ONE_DRIVE_DIR = settings.ONE_DRIVE_DIR
print(f"{ONE_DRIVE_DIR=}")

---------

### Редактирование, поиск и замена в txt файлах из ONE_DRIVE_DIR (файлы с заданием):

#### Задать строку поиска:

In [ ]:
search_text = "Себідин"

#### Задать словарь поиска и замены:

In [2]:
# При непустом участвует в поиске по ключу или замене ключа на значение словаря.
replacer_dict = {
        "Себідин таблетки д/розсмок. №20": ""
        # 'АзитроСандоз - Порошок для оральної суспензії.': 'Азитро Сандоз пор. д/п сусп',
        # 'Окомістин  - розчин': 'Окомістин  - розчин',
    }

#### Задать значение флага для замены. Если False - только поиск, True - замена согласно **replacer_dict**:

In [ ]:
REPLACE = False

#### Выполнить поиск и/или замену от заданных аргументов:

In [ ]:
search_replace(search_text, replacer_dict, REPLACE)

---------------------

### Создание и просмотр списка препаратов для скрапинга:

In [ ]:
df = transform_to_df(settings.ONE_DRIVE_DIR)
DRUG_LIST = sorted(df["drug"].unique())
print(f"{len(DRUG_LIST)=}")

#### Вывести DRUG_LIST:

In [ ]:
pprint(DRUG_LIST)

--------------

### **Скрапинг:**
* Скрапится домен DOMAIN_URL
* Соскрапленные html файлы сохраняются в папке SOURCE_HTML_DIR

In [ ]:
print(f"{DOMAIN_URL=}")
print(f"{SOURCE_HTML_DIR=}")

In [ ]:
# версия Chrome driver
VERSION_MAIN = 121

##### **Перед началом скрапинга желательно очистить SOURCE_HTML_DIR !**

In [ ]:
print("SCRAPING:")
REFUSED_URLS = scrape_pages(DOMAIN_URL, SOURCE_HTML_DIR, DRUG_LIST, VERSION_MAIN)
print("="*120)

#### Просмотреть не соскрапленные urls:

In [ ]:
print("REFUSED URLS:")
print(f"{len(REFUSED_URLS)=}")
pprint(REFUSED_URLS)

--------------

### **Парсинг:**
* Парсятся файлы из папки SOURCE_HTML_DIR **Предварительно проверить SOURCE_HTML_DIR !**
* Распарсинные данные сохраняются в файле PARSED_DATA_PATH

In [ ]:
print(f"{SOURCE_HTML_DIR=}")
print(f"{PARSED_DATA_PATH=}")

#### Процесс парсинга:

In [ ]:
print(f"{len(DRUG_LIST)=}")
df, refused_url, unused_drugs = parse_pages(SOURCE_HTML_DIR, DRUG_LIST)
df.to_csv(PARSED_DATA_PATH, index=False)
print("=" * 120)

#### Процесс парсинга возвращает:
* df - таблица где колонки это названия полей которые удалось спарсить
* refused_url - препараты файлы которых по какимто причинам не распарсились
* unused_drugs - препараты для которых не нашлось соскрепленных файлов в source_page_dir

In [ ]:
print("=" * 120)
print("Refused urls:")
pprint(refused_url)
print("=" * 120)

print("Unused drugs:")
pprint(unused_drugs)
print("=" * 120)

--------------

### Формирование списков симптомов по диагнозам (Не обязательно при наличии файла):

* Формирует таблицу соспоставления индекса/номера симптома к описанию симптома и сохраняет по пути SYMPTOMS_PATH

* Т. как данные берутся из основного задания settings.TASK_DIR то при наличии таблицы SYMPTOMS_PATH этот процесс можно пропустить

#### Процесс создания таблицы:

In [ ]:
dfs = create_symptoms_dfs(settings.TASK_DIR)
for diagnosis, df in dfs.items():
    print(f"\n{diagnosis}")
    print("-"*120)
    print(*df.to_records(index=False), sep="\n")

print("\n")
if not is_open_file(SYMPTOMS_PATH):
    dfs_to_excel(SYMPTOMS_PATH, dfs, highlighted_columns=["symptom"], massage=f"saved at: '{SYMPTOMS_PATH}'")

-----------

### **Создание DataFrames для загрузки в базу данных:**

* Созданные таблицы сохраняются в excel файле TASK_DFS_PATH

In [2]:
symptoms_dfs = read_excel(SYMPTOMS_PATH)
parsed_drugs_df = pd.read_csv(PARSED_DATA_PATH)
task_df = transform_to_df(settings.ONE_DRIVE_DIR)
dfs, not_found_drugs = create_task_dfs(task_df, parsed_drugs_df, symptoms_dfs)

print("=" * 120)
if not is_open_file(TASK_DFS_PATH):
    dfs_to_excel(TASK_DFS_PATH, dfs, highlighted_columns=["drug", "product_name", "Спосіб застосування та дози"], massage=f"dfs saved: '{TASK_DFS_PATH}'")

Not found drug:'Актифед розчин д/перор. заст. по 100 мл у флак.' in diagnosis:'Менингококковая инфекция'
Not found drug:'Актифед таблетки №12' in diagnosis:'Менингококковая инфекция'
dfs saved: 'D:\PycharmProjects\drugscraping\apteka911\data\api_drugs.xlsx'


#### Печать результирующих таблиц:

In [ ]:
pprint(dfs)